Files conversion
================

Imports

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [2]:
import pims
from pims import *

In [3]:
import cv2
import os

In [4]:
from pathlib import *
current_dir = Path.cwd()
current_dir

PosixPath('/data/sources/microscope_video_conversion/notebooks')

In [5]:
#input_folder = Path("/") / "data" / "IOAN" / "INPUT"
#output_folder = Path("/") / "data" / "IOAN" / "OUTPUT"
input_folder = Path("/media/antonkulaga/Elements/IOAN/PP")
output_folder = Path("/data/IOAN/PP_CONVERTED")

if(not output_folder.exists()):
    output_folder.mkdir()

In [6]:
def convert(from_file: Path, to_file: Path):
    worm = pims.Bioformats(from_file, java_memory='1024m')
    frame = worm[0]
    height, width = frame.shape
    fps = 12.0
    video = cv2.VideoWriter(str(to_file.absolute()), cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height), False)    
    for image in worm:
        image = cv2.normalize(image, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)
        video.write(image)
    print(f"{to_file} successfully converted!")
    return to_file

Processing the directory tree with conversion
=============================================

In [7]:
def process(folder: Path, output: Path):
    print(folder)
    output.mkdir(exist_ok=True)
    for f in folder.iterdir():
        if f.is_dir():
            od: Path = output / f.name
            od.mkdir(exist_ok=True)                        
            process(f, od)
        else:
            if "zvi" in f.name:
                to = output / (f.stem+'.mp4')
                print(f"converting {f.absolute()} to {to}")
                convert(f, to)
    return folder

In [ ]:
process(input_folder, output_folder)
print("All conversions finished!")

/media/antonkulaga/Elements/IOAN/PP
/media/antonkulaga/Elements/IOAN/PP/EXP1
/media/antonkulaga/Elements/IOAN/PP/EXP1/DAY1
/media/antonkulaga/Elements/IOAN/PP/EXP1/DAY1/ife-2
converting /media/antonkulaga/Elements/IOAN/PP/EXP1/DAY1/ife-2/w10_150x.zvi to /data/IOAN/PP_CONVERTED/EXP1/DAY1/ife-2/w10_150x.mp4


/opt/miniconda3/envs/microscope_video_conversion/lib/python3.8/site-packages/pims/bioformats.py:408: UserWarning: Due to an issue with JPype 0.6.0, reading is slower. Please consider upgrading JPype to 0.6.1 or later.
  warn('Due to an issue with JPype 0.6.0, reading is slower. '


/data/IOAN/PP_CONVERTED/EXP1/DAY1/ife-2/w10_150x.mp4 successfully converted!
converting /media/antonkulaga/Elements/IOAN/PP/EXP1/DAY1/ife-2/w11_150x.zvi to /data/IOAN/PP_CONVERTED/EXP1/DAY1/ife-2/w11_150x.mp4
